In [33]:
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio import SeqIO
import random

import sys
sys.path.append('..')
from utils.pdb import seq_from_structure
import requests
import re

In [36]:
# path to FULL msa stockholm file
msa_path = '/nfshomes/vla/cmsc702-protein-lm/PF00011_full'
msa = AlignIO.read(msa_path, 'stockholm')

msa

<<class 'Bio.Align.MultipleSeqAlignment'> instance (43064 records of length 690) at 7f0c1beb0f70>

In [37]:
refs = []
for rec in msa:
    if rec.dbxrefs:
        refs.append((rec.name, rec.dbxrefs))

len(refs)

23

In [38]:
refs

[('HSPS_METJA',
  ['PDB; 1SHS E; 46-147;',
   'PDB; 1SHS C; 46-147;',
   'PDB; 1SHS B; 46-147;',
   'PDB; 4I88 C; 46-147;',
   'PDB; 1SHS A; 46-147;',
   'PDB; 4I88 G; 46-147;',
   'PDB; 1SHS G; 46-147;',
   'PDB; 4I88 H; 46-147;',
   'PDB; 4ELD B; 60-161;',
   'PDB; 4I88 D; 46-147;',
   'PDB; 4ELD A; 60-161;',
   'PDB; 4I88 A; 46-147;',
   'PDB; 1SHS F; 46-147;',
   'PDB; 1SHS D; 46-147;',
   'PDB; 4I88 B; 46-147;',
   'PDB; 1SHS H; 46-147;',
   'PDB; 4I88 F; 46-147;',
   'PDB; 4I88 E; 46-147;']),
 ('Q9RTR5_DEIRA', ['PDB; 4FEI A; 55-147;']),
 ('HS25P_ARATH',
  ['PDB; 5NMS F; 87-184;',
   'PDB; 5NMS A; 87-184;',
   'PDB; 5NMS B; 87-184;',
   'PDB; 5NMS I; 87-184;',
   'PDB; 5NMS C; 87-184;',
   'PDB; 5NMS H; 87-184;',
   'PDB; 5NMS G; 87-184;',
   'PDB; 5NMS J; 87-184;',
   'PDB; 5NMS L; 87-184;',
   'PDB; 5NMS E; 87-184;',
   'PDB; 5NMS D; 87-184;',
   'PDB; 5NMS K; 87-184;']),
 ('Q7JP52_CAEEL', ['PDB; 7PE3 B; 24-119;', 'PDB; 7PE3 A; 48-143;']),
 ('HSPB6_HUMAN',
  ['PDB; 4JUS G; 66-14

In [82]:
pdb_id = "3L1E"

In [83]:
# path to pdb file
pdb_path = '/nfshomes/vla/cmsc702-protein-lm/data/pf00011/pdb3l1e.ent'


parser = PDBParser()
structure = parser.get_structure(pdb_id, pdb_path)
structure

<Structure id=3L1E>

In [84]:
pdb_sequence = seq_from_structure(structure)
pdb_sequence

'SGISEVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFSGPKIPSGVDAGHSERAIPVSR'

In [85]:
len(pdb_sequence)

105

In [86]:
pdb_uniprot_info = requests.get('https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/' + pdb_id).json()[pdb_id]['UniProt']
key_id = list(pdb_uniprot_info.keys())[0]

pdb_name = pdb_uniprot_info[key_id]['name']
pdb_name  # Uniprot entry name

KeyError: '3L1E'

In [87]:
pdb_name = 'CRYAA_BOVIN'

In [88]:
# find sequence corresponding to PDB in the MSA
# MSA record name contains uniprot residue range of sequence that is aligned,
# so we only return the record if it contains our query start and end range
def find_seq_in_MSA(msa, target_name, start=-1, end=-1):

    records = []
    for record in msa:

        rec_split = record.id.split('/')

        rec_name = rec_split[0]
        rec_range = rec_split[1].split('-')
        rec_start = int(rec_range[0])
        rec_end = int(rec_range[1])

        if rec_name == target_name:
            records.append(record)

            
    return records

In [89]:
records = find_seq_in_MSA(msa, pdb_name)
records

[SeqRecord(seq=Seq('------------------------------------------------------...---'), id='CRYAA_BOVIN/63-162', name='CRYAA_BOVIN', description='CRYAA_BOVIN/63-162', dbxrefs=['PDB; 3L1F A; 63-162;', 'PDB; 3L1E A; 63-162;'])]

In [90]:
selected_record = records[0]
selected_record

SeqRecord(seq=Seq('------------------------------------------------------...---'), id='CRYAA_BOVIN/63-162', name='CRYAA_BOVIN', description='CRYAA_BOVIN/63-162', dbxrefs=['PDB; 3L1F A; 63-162;', 'PDB; 3L1E A; 63-162;'])

In [91]:
seq_range = selected_record.dbxrefs[1].split(';')[2].split('-')
seq_range


[' 63', '162']

In [92]:
# SOMETIMEs this mapping is shifted down one WHY
seq_start = int(seq_range[0]) - 1
seq_end = int(seq_range[1])
seq_start,seq_end

(62, 162)

In [93]:
gapped_str = str(selected_record.seq)
ungapped_seq = gapped_str.replace('-', '')
# raw sequence has same amount of residues as the uniprot range specified
# assert len(ungapped_seq) == uniprot_ref_end - uniprot_ref_start + 1
ungapped_seq

'eVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLsADGMLTFSGPKIPSGVDagHSERAIPVs'

In [94]:
pdb_sequence

'SGISEVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFSGPKIPSGVDAGHSERAIPVSR'

In [95]:
pdb_sequence.find('eVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLsADGMLTFSGPKIPSGVDagHSERAIPVs'.upper())

4

In [96]:
pdb_sequence[4:]

'EVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFSGPKIPSGVDAGHSERAIPVSR'

In [103]:
seq_start = 4
seq_end = len(pdb_sequence) - 1

In [111]:
ungapped_seq.upper()  

'EVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFSGPKIPSGVDAGHSERAIPVS'

In [112]:
pdb_sequence[seq_start:seq_end]

'EVRSDRDKFVIFLDVKHFSPEDLTVKVQEDFVEIHGKHNERQDDHGYISREFHRRYRLPSNVDQSALSCSLSADGMLTFSGPKIPSGVDAGHSERAIPVS'

In [113]:
# make sure this doesnt fail
assert ungapped_seq.upper() == pdb_sequence[seq_start:seq_end]

In [114]:
msa_start = len(gapped_str) - len(gapped_str.lstrip('-'))
msa_start # inclusive msa col to start at

109

In [115]:
msa_end = len(gapped_str) - (len(gapped_str) - len(gapped_str.rstrip('-')))
msa_end # exclusive msa col to end at

635

In [116]:
meta = {}
meta['pfam_id'] = 'PF00011'
meta['pdb_id'] = pdb_id
meta['pdb_name'] = pdb_name
meta['seq_range'] = (seq_start, seq_end)   # range of pdb sequence that MSA entry refers to [start,end) python indexing
meta['msa_range'] = (msa_start,msa_end)    # range of msa cols to crop msa to to include the MSA entry of interest [start,end) python indexing
meta['uniprot_range'] = (59,163)           # Uniprot residue range covered by the entire pdb structure [start,end] (for record purposes), find on https://www.ebi.ac.uk/interpro/structure/PDB/

meta


{'pfam_id': 'PF00011',
 'pdb_id': '3L1E',
 'pdb_name': 'CRYAA_BOVIN',
 'seq_range': (4, 104),
 'msa_range': (109, 635),
 'uniprot_range': (59, 163)}

In [117]:
import json
import os

output_dir = '../data/pf00011'

with open(os.path.join(output_dir, f'meta_{pdb_name}.json'), 'w') as f:
    json.dump(meta, f)

In [118]:
# Parameters
msa_path = '/nfshomes/vla/cmsc702-protein-lm/PF00011_full' # path to FULL msa stockholm file
N = 10000 # number of sequences to select
L = msa_start # left msa index
R = msa_end # right msa index 
output_path = f'/nfshomes/vla/cmsc702-protein-lm/data/pf00011/PF00011_{N}_msa.faa' # output path for new fasta file
threshold = 0 # minimal gap ratio to accept [0-1]

In [119]:
random.seed(42)

if threshold == None:
    threshold = 0
try:
    seen = set(selected_record[L:R].seq)
    records = []

    def gap_ratio(seq):
        return len(seq.replace('-',''))/len(seq)

    for rec in SeqIO.parse(msa_path, 'stockholm'): 
        if rec[L:R].seq not in seen:
            if gap_ratio(rec[L:R].seq) >= threshold:
                seen.add(rec[L:R].seq)
                records.append(rec[L:R])     
except:
    print('could not parse file -> not in fasta format')



random.shuffle(records) 
records = [selected_record[L:R]] + records

SeqIO.write(records[0:N], output_path, "fasta")



10000

In [120]:
records[0]

SeqRecord(seq=Seq('e-VRSDRDKF-V-I-F-L-D--V----K---H----F----S-------P----...V-s'), id='CRYAA_BOVIN/63-162', name='CRYAA_BOVIN', description='CRYAA_BOVIN/63-162', dbxrefs=[])

In [121]:
records[0].seq[-1]

's'

In [122]:
SeqIO.write(records[0:N], output_path, "fasta")

10000